# dHybridR M03 Analysis
#### Here we analyze a specific simulation, assuming that FPC data has already been generated

In [4]:
#Need this to add libraries in parent folder
import sys
sys.path.append("..")

In [5]:
import lib.analysis as anl 
import lib.array_ops as ao
import lib.data_h5 as dh5
import lib.data_netcdf4 as dnc
import lib.fpc as fpc
import lib.frametransform as ft
import lib.metadata as md

import lib.plot.oned as plt1d
import lib.plot.twod as plt2d
import lib.plot.debug as pltdebug
import lib.plot.fourier as pltfr
import lib.plot.resultsmanager as rsltmng
import lib.plot.velspace as pltvv

import os
import math
import numpy as np

## Load data

In [ ]:
# load analysis file
analysisflnm = '../analysisinputs/m03input.txt'
path,resultsdir,vmax,dv,numframe,dx,xlim,ylim,zlim = anl.analysis_input(flnm = analysisflnm)


In [ ]:
#load fields
dfields = dh5.field_loader(path=path,num=numframe)

#load flow data
dflow = dh5.flow_loader(path=path,num=numframe)


In [ ]:
#check that xlim ylim and zlim are not none
#if so, use boundaries of box
cellsizexx = dfields['ex_xx'][1]-dfields['ex_xx'][0]
cellsizeyy = dfields['ex_yy'][1]-dfields['ex_yy'][0]
cellsizezz = dfields['ex_zz'][1]-dfields['ex_zz'][0]

if(xlim is None):
    xlim = [dfields['ex_xx'][0]-cellsizexx/2.,dfields['ex_xx'][-1]+cellsizexx/2.]
if(ylim is None):
    ylim = [dfields['ex_yy'][0]-cellsizeyy/2.,dfields['ex_yy'][-1]+cellsizeyy/2.]
if(zlim is None):
    zlim = [dfields['ex_zz'][0]-cellsizeyy/2.,dfields['ex_zz'][-1]+cellsizeyy/2.]

In [ ]:
# load particle data
dparticles = dh5.read_box_of_particles(path_particles, numframe, xlim[0], xlim[1], ylim[0], ylim[1], zlim[0], zlim[1])

In [ ]:
all_dfields = dh5.all_dfield_loader(path=path, verbose=False)

### Lorentz transform fields and shift frame of flow

In [ ]:
vshock, xshockvals = ft.shock_from_ex_cross(all_dfields)

In [ ]:
## (Optional) Check if shock was tracked well. We use the back half in time to fit to a line
#plt1d.time_stack_line_plot(all_dfields,'ex',pts = xshockvals)

In [ ]:
#loretnz transform fields
dfields = ft.lorentz_transform_vx(dfields,vshock)
_fields = []
for k in range(0,len(all_dfields['dfields'])):
    _fields.append(ft.lorentz_transform_vx(all_dfields['dfields'][k],vshock))
all_dfields['dfields'] = _fields

#tranform flow
dflow = ft.transform_flow(dflow,vx):

In [ ]:
# (Optional) plot all the lorentz transformed fields
plt1d.plot_all_fields(dfields, axis='_xx', xxindex = 0, yyindex = 0, zzindex = 0)

In [ ]:
# (Optional) plot flow in the etimated frame of the shock
plt1d.plot_all_flow(dflow, axis='_xx', xxindex = 0, yyindex = 0, zzindex = 0)

### Load FPC data

In [ ]:
ncflnm = '../M03results/m03.nc'
Hist_in, CEx_in, CEy_in, CEz_in, vx_in, vy_in, vz_in, x_in, enerCEx_in, enerCEy_in, enerCEz_in, Vframe_relative_to_sim, metadata_in, params_in = dnc.load3Vnetcdf4(ncflnm)

### Look for signature bounds

In [ ]:
startval = 17.5#lower bound for sda metadata = 1 (di)
endval = 21.5 #upper bound for sda metadata = 1 (di)

In [ ]:
#option one, use Ex(x,y=0,z=0) to find SDA bounds. Should range from the start of the ramp to the first zero crossing


yyindex = 0
zzindex = 0
plt1d.plot_field(dfields, 'ex', axis='_xx', yyindex = yyindex, zzindex = zzindex, axvx1 = startval, axvx2 = endval)

In [ ]:
#option two (better but harder), look at the FPC signatures and see when the shock starts and stops

#Plot FPCs and dist funcs at start val-------------------------------------------------------------
#grab index of starval
i = ao.find_nearest(dfields['ex_xx', starval)

#Project onto 2d axis
H_xy = array_3d_to_2d(Hist[i],'xy')
H_xz = array_3d_to_2d(Hist[i],'xz')
H_yz = array_3d_to_2d(Hist[i],'yz')
CEx_xy = array_3d_to_2d(CEx[i],'xy')
CEx_xz = array_3d_to_2d(CEx[i],'xz')
CEx_yz = array_3d_to_2d(CEx[i],'yz')
CEy_xy = array_3d_to_2d(CEy[i],'xy')
CEy_xz = array_3d_to_2d(CEy[i],'xz')
CEy_yz = array_3d_to_2d(CEy[i],'yz')
CEz_xy = array_3d_to_2d(CEz[i],'xy')
CEz_xz = array_3d_to_2d(CEz[i],'xz')
CEz_yz = array_3d_to_2d(CEz[i],'yz')

#plot
pltvv.plot_cor_and_dist_supergrid(vx, vy, vz, vmax,
                                H_xy, H_xz, H_yz,
                                CEx_xy,CEx_xz, CEx_yz,
                                CEy_xy,CEy_xz, CEy_yz,
                                CEz_xy,CEz_xz, CEz_yz)

In [ ]:
#Plot FPCs and dist funcs at end val--------------------------------------------------------------
#grab index of starval
i = ao.find_nearest(dfields['ex_xx', endval)

#Project onto 2d axis
H_xy = array_3d_to_2d(Hist[i],'xy')
H_xz = array_3d_to_2d(Hist[i],'xz')
H_yz = array_3d_to_2d(Hist[i],'yz')
CEx_xy = array_3d_to_2d(CEx[i],'xy')
CEx_xz = array_3d_to_2d(CEx[i],'xz')
CEx_yz = array_3d_to_2d(CEx[i],'yz')
CEy_xy = array_3d_to_2d(CEy[i],'xy')
CEy_xz = array_3d_to_2d(CEy[i],'xz')
CEy_yz = array_3d_to_2d(CEy[i],'yz')
CEz_xy = array_3d_to_2d(CEz[i],'xy')
CEz_xz = array_3d_to_2d(CEz[i],'xz')
CEz_yz = array_3d_to_2d(CEz[i],'yz')

#plot
pltvv.plot_cor_and_dist_supergrid(vx, vy, vz, vmax,
                                H_xy, H_xz, H_yz,
                                CEx_xy,CEx_xz, CEx_yz,
                                CEy_xy,CEy_xz, CEy_yz,
                                CEz_xy,CEz_xz, CEz_yz)

In [ ]:
#set metadata using add Metadata script
cmd = 'python3 ../scripts/addMetadata.py ' + analysisflnm + ' ' + ncflnm + ' ' + str(startval) + ' ' + str(endval)
os.system(cmd)

### Look for shock rippling

#### Make pmesh of fields

In [ ]:
#plot xy plane at z=0 
plt2d.make_field_pmesh(dfields,'bz','xy',flnm = '',takeaxisaverage=False, xxindex=float('nan'), yyindex=float('nan'), zzindex=0, xlimmin=None,xlimmax=None)

In [ ]:
#zoom in on potential ripple xy plane at z=0
plt2d.make_field_pmesh(dfields,'bz','xy',flnm = '',takeaxisaverage=False, xxindex=float('nan'), yyindex=float('nan'), zzindex=0, xlimmin=startval,xlimmax=endval)

#### Make stacked line plot

In [7]:
plt1d.plot_stack_field_along_x(dfields,'bz','_yy',yyindex=0,zzindex=0,xlow=starval,xhigh=endval)

NameError: name 'dfields' is not defined

### Diagnose corrugation instability
#### look at number of significant wave modes in fourier space
#### still WIP

In [ ]:
shockval = 15.0 #di, some slice in the ramp of the shock
i = ao.find_nearest(dfields['ex_xx', shockval)
pltfr.plot_fft_norm(dfields,'bz','yz',flnm = '',takeaxisaverage=False, xxindex=i, yyindex=float('nan'), zzindex=float('nan'), plotlog = True, xaxislim=None, yaxislim=None)